In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats, interpolate
import scipy
import warnings
import datetime as dt
import landau

import importlib

# local imports
from lib import glob
from lib import plot
from lib import branches
from lib.constants import *

In [ ]:
importlib.reload(glob)
importlib.reload(plot)
importlib.reload(branches)

In [ ]:
savedir = "/icarus/app/users/gputnam/calib/plots2/"
dosave=False

In [ ]:
# Get external information on run timing
run_times = {}
with open("/icarus/app/users/gputnam/calib/rundata") as f:
    for line in f:
        dat = line.split(" ")
        run_times[int(dat[0])] = dt.datetime.strptime(dat[1].rstrip("\n"), "%Y-%m-%dT%H:%M:%S").date()

In [ ]:
# EXTERNAL INPUT: The drift window in each TPC
tcathode = 2330.188679245283 + tanode

tcathode_EE = tcathode
tcathode_EW = tcathode
tcathode_WE = tcathode
tcathode_WW = tcathode

In [ ]:
#data = ntuples.dataframe(nproc="auto")
perhit_df = pd.read_hdf('/icarus/data/users/gputnam/calib/mc/MCP2021G2/etau.hdf5')

In [ ]:
perhit_df["true_dqdx"] = perhit_df.true_nelec

del perhit_df["true_nelec"]

In [ ]:
perhit_df

In [ ]:
perhit_df["thit"] = (perhit_df.time * tick_period - perhit_df.ccross_t0 - tanode*tick_period) / 1000.

In [ ]:
cosphiz_E = np.sqrt(3)/ 2.
cosphiz_W = cosphiz_E

sinphiz_E = -0.5
sinphiz_W = 0.5

transverse_D = 8.8e-6 #cm^2 / us
driftV = 0.15902 # cm/us

wire_pitch = 0.3 # cm

perhit_df["cosgamma"] = cosphiz_W * perhit_df.dirz + sinphiz_W*perhit_df.diry
perhit_df.loc[perhit_df.tpcE, "cosgamma"] = (cosphiz_E * perhit_df.dirz + sinphiz_E*perhit_df.diry)[perhit_df.tpcE]

perhit_df["costh_u"] = np.cos(np.arctan2(perhit_df.dirx, perhit_df.cosgamma))

perhit_df["tick_window"] = np.abs(perhit_df.dirx)*((wire_pitch + 4*np.sqrt(2*np.maximum(perhit_df.thit, 0.)*transverse_D)) / np.abs(perhit_df.cosgamma)) \
        / (driftV) / (tick_period*1e-3)

In [ ]:
cals = [0.011533881380744694,
 0.012496236276693981,
 0.013478337131491012,
 0.013978330190774504,
 0.014171785821236378,
 0.014264403089615416,
 0.0143044315404323,
 0.014335080321068444,
 0.014359347972840403,
 0.014364720533883569,
 0.014369446126679638,
 0.014382243111430106,
 0.014383512011838778,
 0.01439945144350376]

ws = [3.25,
 3.75,
 4.25,
 4.75,
 5.25,
 5.75,
 6.25,
 6.75,
 7.25,
 7.75,
 8.25,
 8.75,
 9.25,
 9.75]

ws.insert(0, 0.)
cals.insert(0, cals[0])

ws.append(1e100)
cals.append(cals[-1])

window_to_cal = interpolate.interp1d(ws, cals)#, fill_value=(ws[0], ws[-1]) )

perhit_df["dqdx_widthcor"] = perhit_df.dqdx / window_to_cal(perhit_df.width)

In [ ]:
if "start_width" in perhit_df.columns:
    del perhit_df["start_width"]
    
start_width = perhit_df.loc[perhit_df.time.groupby(level=0).idxmin()].width
start_width.name = "start_width"
start_width = start_width.droplevel(1)
perhit_df = perhit_df.join(start_width)

In [ ]:
perhit_df

In [ ]:
xs = np.linspace(-10, 10, 10001)
ys = landau.landau.landau(xs, 0, 1)
plt.plot(xs, ys)
plt.vlines(0, 0, np.max(ys))

In [ ]:
cnames = "EW"
cryostats = [0,1]

In [ ]:
t0=0.
def exp(t, *p):
    A,tau = p
    return A*np.exp(-(t - t0)/tau)

def gaus(x, *p):
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2 / sigma**2)

def landau_gaus(X, *p):
    A, mpv, eta, sigma = p
    sigma = np.minimum(sigma, 100*eta)
    return landau.landau.gauss_landau(X, mpv, eta, sigma, A)

tmax = a2c_dist / 0.1574
binx = np.linspace(0, tmax, 21)

def get_etau(time, dqdx, bx, by):
    
    N, xbin, ybin = np.histogram2d(time, dqdx, bins=[bx, by])
    ycenter = (ybin[1:] + ybin[:-1]) / 2.
    xcenter = (xbin[1:] + xbin[:-1]) / 2.

    sample_mean = np.sum(ycenter*N, axis=-1) / np.sum(N, axis=-1)

    #mean_subtract = np.repeat(sample_mean.reshape((sample_mean.size,1)), ycenter.size, axis=1)
    #stddev = np.sqrt(np.sum(N*(ycenter-mean_subtract)**2, axis=-1)) / np.sqrt(np.sum(N, axis=-1))
    #stderr = stddev / np.sqrt(np.sum(N, axis=-1))
    
    mean = []
    mean_err = []
    fits = []
    for i in range(len(xbin)-1):
        Nfit = N[i, :]
        # p0 = [np.max(Nfit), sample_mean[i], np.sqrt(sample_mean[i])] 
        p0 = [np.max(Nfit)*40, sample_mean[i], np.sqrt(sample_mean[i]), np.sqrt(sample_mean[i])] 
        popt, perr = curve_fit(landau_gaus, ycenter, Nfit, p0=p0, sigma=np.sqrt(Nfit), maxfev=1_000_000)
        mean.append(popt[1])
        mean_err.append(np.sqrt(np.diag(perr))[1])
        fits.append(lambda x, popt=popt: landau_gaus(x, *popt))
    mean = np.array(mean)
    mean_err = np.array(mean_err)
    
    where_fit = ~np.isnan(xcenter) & (xcenter > 200) & (xcenter < 750) & np.isfinite(mean_err)
    p0 = [ycenter[ycenter.size//2], 3.5e3]
    popt, perr = curve_fit(exp, xcenter[where_fit], mean[where_fit], p0=p0, sigma=mean_err[where_fit])
    
    return xcenter, mean, mean_err, popt, perr, fits

In [ ]:
etau_Es = [0, 0]
etau_Ws = [0, 0]

etau_err_Es = [0, 0]
etau_err_Ws = [0, 0]

ifig = 0

biny = np.linspace(200, 1400, 35)

for c in cryostats:
    cname = cnames[c]
    for tpc in "EW":
        when = (perhit_df.tpcE == (tpc =="E")) & (perhit_df.cryostat==c) & (perhit_df.pitch < 1.) #& (perhit_df.start_width > 6)

        plt.figure(ifig)
        N, xbin, ybin,_ = plt.hist2d(perhit_df.thit[when], perhit_df.dqdx[when], bins=[binx, biny])
        cbar = plt.colorbar()
        cbar.set_label("Entries")
 
        times, means, errs, popt, perr, fits = get_etau(perhit_df.thit[when], perhit_df.dqdx[when], binx, biny)
    
        plt.errorbar(times, means, yerr=errs, color="r", label="Data M.P.V.")

        plt.xlabel("Deposition Time [us]")
        plt.ylabel("dQ/dx [ADDC/cm]")
        plt.title("Cryostat %s TPC %s Plane 2" % (cname, tpc))

        ifig += 1
        xcenter = (binx[1:] + binx[:-1]) /2.
        ycenter = (biny[1:] + biny[:-1]) / 2.
        plt.plot(xcenter, exp(times, *popt), label="Exp Fit", color="w")
        plt.legend()
        plt.xlabel("Deposition Time [us]")
        plt.ylabel("Median dQ/dx [ADC/cm]")
        plt.title("MC Cryostat %s TPC %s Plane 2" % (cname, tpc))

        etau = (popt[1]/1e3)
        etau_err = np.sqrt(np.diag(perr))[1]/1e3
        print(etau, etau_err)
        plt.text(0.05, 0.05, 
                 "Electron Lifetime: %.2f [ms]" % etau, color="white", transform=plt.gca().transAxes, fontsize=14)

        if tpc == "E": 
            etau_Es[c] = (etau)
            etau_err_Es[c] = (etau_err)
        else: 
            etau_Ws[c] = (etau)
            etau_err_Ws[c] = (etau_err)
            
        #Ntime = np.sum(N, axis=1)
        if False:
            for i in range(N.shape[0]):
                plt.figure(ifig)
                ifig += 1

                _ = plt.hist(ycenter, bins=biny, weights=N[i, :])
                yplot = np.linspace(ycenter[0], ycenter[-1], 1000)
                plt.plot(yplot, fits[i](yplot), color="r")
                plt.vlines(means[i], 0, np.max(N[i, :]), color="r")

        if dosave: plt.savefig(savedir + "etau_run%i_tpc%s%s.png" % (r, cname, tpc))

In [ ]:
etau_Es = [0, 0]
etau_Ws = [0, 0]

etau_err_Es = [0, 0]
etau_err_Ws = [0, 0]

ifig = 0
biny_elec = np.linspace(2e4, 9e4, 101)

for c in cryostats:
    cname = cnames[c]
    for tpc in "EW":
        when = (perhit_df.tpcE == (tpc =="E")) & (perhit_df.cryostat==c) & (perhit_df.pitch < 1.) #& (perhit_df.start_width > 6)

        plt.figure(ifig)
        N, xbin, ybin,_ = plt.hist2d(perhit_df.thit[when], perhit_df.dqdx_widthcor[when], 
                                     bins=[binx, biny_elec])
        cbar = plt.colorbar()
        cbar.set_label("Entries")
 
        times, means, errs, popt, perr, fits = get_etau(perhit_df.thit[when], 
                                                  perhit_df.dqdx_widthcor[when], bx=binx, by=biny_elec)
    
        #ifig += 1
        #plt.figure(ifig)
        plt.errorbar(times, means, yerr=errs, color="r", label="Data M.P.V.")

        plt.xlabel("Deposition Time [us]")
        plt.ylabel("Truth-Corrected Median dQ/dx [#elec/cm]")

        ifig += 1
        xcenter = (binx[1:] + binx[:-1]) /2.
        ycenter = (biny[1:] + biny[:-1]) / 2.
        plt.plot(xcenter, exp(times, *popt), label="Exp Fit", color="w")
        plt.legend()
        plt.title("MC Cryostat %s TPC %s Plane 2" % (cname, tpc))

        etau = (popt[1]/1e3)
        etau_err = np.sqrt(np.diag(perr))[1]/1e3
        print(etau, etau_err)
        plt.text(0.05, 0.05, 
                 "Electron Lifetime: %.2f [ms]" % etau, color="white", transform=plt.gca().transAxes, fontsize=14)

        if tpc == "E": 
            etau_Es[c] = (etau)
            etau_err_Es[c] = (etau_err)
        else: 
            etau_Ws[c] = (etau)
            etau_err_Ws[c] = (etau_err)

        if dosave: plt.savefig(savedir + "etau_run%i_tpc%s%s.png" % (r, cname, tpc))

In [ ]:
etau_Es = [0, 0]
etau_Ws = [0, 0]

etau_err_Es = [0, 0]
etau_err_Ws = [0, 0]

ifig = 0

for c in cryostats:
    cname = cnames[c]
    for tpc in "EW":
        when = (perhit_df.tpcE == (tpc =="E")) & (perhit_df.cryostat==c) #& (perhit_df.start_width > 6)

        plt.figure(ifig)
        N, xbin, ybin,_ = plt.hist2d(perhit_df.thit[when], (perhit_df.true_dqdx)[when], 
                                     bins=[binx, biny_elec])
        cbar = plt.colorbar()
        cbar.set_label("Entries")
 
        times, means, errs, popt, perr, fits = get_etau(perhit_df.thit[when], 
                                                  (perhit_df.true_dqdx)[when], bx=binx, by=biny_elec)
    
        plt.figure(ifig)
        plt.errorbar(times, means, yerr=errs, color="r", label="Data M.P.V.")

        plt.xlabel("Deposition Time [us]")
        plt.ylabel("dQ/dx [ADDC/cm]")
        plt.title("Cryostat %s TPC %s Plane 2" % (cname, tpc))

        ifig += 1
        xcenter = (binx[1:] + binx[:-1]) /2.
        plt.plot(xcenter, exp(times, *popt), label="Exp Fit", color="w")
        plt.legend()
        plt.xlabel("Deposition Time [us]")
        plt.ylabel("Median dQ/dx [ADC/cm]")
        plt.title("MC Cryostat %s TPC %s Plane 2" % (cname, tpc))

        etau = (popt[1]/1e3)
        etau_err = np.sqrt(np.diag(perr))[1]/1e3
        print(etau, etau_err)
        plt.text(0.05, 0.05, 
                 "Electron Lifetime: %.2f [ms]" % etau, color="white", transform=plt.gca().transAxes, fontsize=14)

        if tpc == "E": 
            etau_Es[c] = (etau)
            etau_err_Es[c] = (etau_err)
        else: 
            etau_Ws[c] = (etau)
            etau_err_Ws[c] = (etau_err)

        if dosave: plt.savefig(savedir + "etau_run%i_tpc%s%s.png" % (r, cname, tpc))
            
        if True:
            ycenter = (biny_elec[1:] + biny_elec[:-1]) / 2.

            for i in range(N.shape[0]):
                plt.figure(ifig)
                ifig += 1

                _ = plt.hist(ycenter, bins=biny_elec, weights=N[i, :])
                yplot = np.linspace(ycenter[0], ycenter[-1], 1000)
                plt.plot(yplot, fits[i](yplot), color="r")
                plt.vlines(means[i], 0, np.max(N[i, :]), color="r")
                
        break
    break

In [ ]:
etau_Es, etau_Ws

In [ ]:
start_widths = np.linspace(3, 7, 41) 

for c in cryostats:
    cname = cnames[c]
    for tpc in "EW":
        etaus = []
        for w in start_widths:
            when = (perhit_df.tpcE == (tpc=="E")) & (perhit_df.cryostat==c) & (perhit_df.start_width > w)
            times, means, errs, popt, perr = get_etau(perhit_df.thit[when], perhit_df.dqdx[when])
            etaus.append(popt[1]/1e3)

        plt.plot(start_widths, etaus, label="TPC " +cname+tpc)
plt.legend()

In [ ]:

binx = np.linspace(3, 5., 11)
biny = np.linspace(400, 1000, 26)

ifig = 0


for c in cryostats:
    cname = cnames[c]
    for tpc in "EW":
        when = (perhit_df.tpcE == (tpc =="E")) & (perhit_df.cryostat==c) &\
            (perhit_df.thit > 100.) & (perhit_df.thit < 200.)

        plt.figure(ifig)
        N, xbin, ybin,_ = plt.hist2d(perhit_df.width[when], perhit_df.dqdx[when], bins=[binx, biny])
        ifig += 1

In [ ]:

binx = np.linspace(3, 10., 15)
biny = np.linspace(400, 1000, 26)

ifig = 0


for c in cryostats:
    cname = cnames[c]
    for tpc in "EW":
        when = (perhit_df.tpcE == (tpc =="E")) & (perhit_df.cryostat==c) &\
            (perhit_df.thit > 800.) & (perhit_df.thit < 850.)

        plt.figure(ifig)
        N, xbin, ybin,_ = plt.hist2d(perhit_df.width[when], perhit_df.dqdx[when], bins=[binx, biny])
        ifig += 1

In [ ]:

binx = np.linspace(3, 10., 15)
biny = np.linspace(400, 1000, 26)

ifig = 0


for c in cryostats:
    cname = cnames[c]
    for tpc in "EW":
        when = (perhit_df.tpcE == (tpc =="E")) & (perhit_df.cryostat==c) &\
            (perhit_df.thit > 800.) & (perhit_df.thit < 850.)

        plt.figure(ifig)
        N, xbin, ybin,_ = plt.hist2d(perhit_df.width[when], perhit_df.dqdx_sumadc[when], bins=[binx, biny])
        ifig += 1

# 